In [13]:
import math

import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import torch
from tqdm import tqdm
import pandas as pd
import math
from sklearn.metrics import accuracy_score

In [2]:
model_name = "LeoLM/leo-mistral-hessianai-7b"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id=tokenizer.eos_token_id

In [4]:
generation_config=GenerationConfig(max_new_tokens=30,
                                    temperature=0.4,
                                    top_p=0.95,
                                    top_k=40,
                                    repetition_penalty=1.2,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )

In [5]:
def prompt_model(prompt):
    input_tokens=tokenizer("Du bist ein hilfreicher assistent, der alle fragen so korrekt wie möglich beantwortet. <BENUTZER>:" + prompt + " <ASSISTENT>:", return_tensors="pt").to(model.device)
    output_tokens=model.generate(**input_tokens, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)[0]
    answer=tokenizer.decode(output_tokens, skip_special_tokens=True)
    return answer

In [6]:
prompt_case = "Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Die Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Beantworte die Frage, indem du die korrekte Antwortmöglichkeit wiedergibst."
prompt_no_case = "Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Die Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Beantworte die Frage, indem du die korrekte Antwortmöglichkeit wiedergibst."

In [7]:
questions_df = pd.read_csv("../staatsexamen/hs1_hs2.csv")
questions_df["Prediction LLM"] = [-1 for i in range(len(questions_df))]
questions_df["Answer LLM"] = [-1 for i in range(len(questions_df))]

In [12]:
for index, row in tqdm(questions_df.iterrows()):
    if type(row["Case"]) == str:
        prompt = prompt_case.replace("<INSERTFALL>",row["Case"])
    else:
        prompt = prompt_no_case
    prompt = prompt.replace("<INSERTFRAGE>",row["Question"])
    for i in range(5):
        prompt = prompt.replace("<INSERTANTWORT" + str(i) + ">",row["Answer "+ str(i+1)].replace(str(i) + ")",""))
    answer = prompt_model(prompt).split("<ASSISTENT>:")[1].split("ENDANTWORT")
    found = False
    for i in range(5):
        if row["Answer "+ str(i+1)].replace(str(i) + ")","").strip().lower() == answer[0].strip().lower():
            found = i
            break
    #print(answer)
    if found != False:
        questions_df.loc[index, "Answer LLM"] = found
    questions_df.loc[index, "Prediction LLM"] = " ENDANTWORT ".join(answer)
print("Values set:", len(questions_df[questions_df["Answer LLM"] != -1]),"/",len(questions_df))


Values set: 144 / 248


In [16]:
questions_df.to_csv("leo_mistral_hessiani_output.csv")

In [14]:
y_pred = questions_df[questions_df["Answer LLM"] != -1]["Answer LLM"]
y_true = questions_df[questions_df["Answer LLM"] != -1]["Correct Answer"]

In [15]:
print("Accuracy:",accuracy_score(y_true, y_pred))

Accuracy: 0.3680555555555556
